In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import FeatureUnion
from sklearn.impute import SimpleImputer, MissingIndicator
import gc
import janestreet
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train_df = train_df[train_df.weight != 0 ]

In [ ]:
train_df.head()

In [ ]:
features = [c for c in train_df.columns if 'feature' in c] + ['date']

X = train_df.loc[:, features].values
y = train_df.loc[:, 'resp'].values

In [ ]:
del train_df

gc.collect()

In [ ]:

transformer = FeatureUnion(
    transformer_list=[
        ('features', SimpleImputer(strategy='mean')),
        ('indicators', MissingIndicator())])

transformer = transformer.fit(X, y)
X = transformer.transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, shuffle=False)

In [ ]:
del X, y

gc.collect()

In [ ]:


clf = LinearRegression(n_jobs=-1)

clf.fit(X_train,y_train)
y_hat = clf.predict(X_train)

y_hat_test = clf.predict(X_test)

print(f'train rmse  {np.sqrt(mean_squared_error(y_hat,y_train))}')
print(f'test rmse  {np.sqrt(mean_squared_error(y_hat_test,y_test))}')



In [ ]:
from sklearn import model_selection
from sklearn.metrics import accuracy_score
kfold = model_selection.KFold(n_splits=10, random_state=0)

scoring = 'accuracy'
results = model_selection.cross_val_score(clf, X_train, y_hat, cv=kfold, scoring=scoring)
print('Accuracy -val set: %.2f%% (%.2f)' % (results.mean()*100, results.std()))